## 크로스 인코더를 이용한 리랭킹
### 크로스 인코더 기반의 리랭킹
크로스 인코더 기반 리랭킹이라는 설명을 하기 위해서는 우선 이중 인코더 (bi‑encoder) 와 크로스 인코더 (cross‑encoder) 방식의 차이를 이해해야 합니다.

검색 시스템의 기본이 되는 이중 인코더는 텍스트를 벡터로 변환하여 의미적 유사도를 계산하는 방식입
니다. 우리가 지금까지 LangChain 의 Chroma와 OpenAIEmbeddings를 통해 사용하던 기본 검색 방
식이 바로 이 이중 인코더입니다. 검색 과정에서 이중 인코더는 질문을 벡터로 변환하고, 데이터베이스의
모든 문서들도 각각 벡터로 변환합니다. 예를 들어 “고양이의 수명” 이라는 질문은 하나의 벡터가 되고,
데이터베이스의 각 문서 조각들도 각자의 벡터를 가지게 됩니다. 그런 다음 질문 벡터와 문서 벡터들 간
의 코사인 유사도를 계산하여 가장 유사한 문서들을 찾아냅니다. 문서의 벡터를 미리 계산해두고 저장할
수 있어 검색 속도가 매우 빠르다는 장점이 있지만, 질문과 문서를 독립적으로 처리하기 때문에 정확도 면
에서는 한계가 있습니다.

반대로 크로스 인코더는 질문과 문서를 따로 벡터화하지 않고 하나의 쌍으로 입력받아 직접적으로 관련성을 판단합니다. 이 방식은 두 텍스트의 관계를 동시에 고려하여 문맥을 더 깊이 이해하기 때문에 더 정
확한 관련성 판단이 가능합니다. 하지만 치명적인 단점이 있습니다. 문서가 1000 개라면 검색어가 들어
올 때마다 1000 번의 연산을 새로 해야 하므로, 실시간 검색 시스템의 첫 단계 검색기로는 사용할 수 없습
니다.

- 여기서 잠깐!! 두 방법 모두 일반적으로 BERT를 사용합니다. 이중 인코더는 BERT를 이용하여 각
각 임베딩 한 후에 유사도를 계산하고, 크로스 인코더는 KorNLI 챕터에서 배웠던 방식처럼
두 개의 텍스트 입력을 동시에 넣으면 스코어가 출력되는 방식입니다.

이런 상황에서 리랭킹이라는 해결책이 등장합니다. 앞으로 우리 코드에서 사용할 BAAI/bgereranker-v2-m3는 크로스 인코더 모델로, 이를 2 단계 검색의 재정렬 단계에서 활용할 것입니다. 먼
저 빠른 이중 인코더로 문서를 일부만 추려낸 다음, 이 적은 수의 문서에 대해서만 크로스 인코더를 적용하는 방식입니다.

실제 예시를 들어보면, “고양이의 수명” 이라는 질문이 들어왔을 때 먼저 이중 인코더로 빠르게 4 개의 관
련 문서를 찾습니다. 그런 다음 BAAI/bge-reranker가 이 4 개의 문서만을 질문과 쌍으로 만들어 더
정교한 관련성을 계산합니다. 이때는 4 쌍만 평가하면 되므로 속도 문제가 없습니다. 이렇게 크로스 인코
더의 정확도와 이중 인코더의 속도를 모두 활용할 수 있는 것이 바로 리랭킹의 핵심입니다. 이러한 이중
필터링 과정을 통해 검색의 정확도를 크게 향상시킬 수 있으며, 이제 이 개념을 실제 코드로 구현해보도록
하겠습니다.


### 2. 패키지 설치 및 OpenAI키 값 설정
랭체인 패키지와 벡터 데이터베이스를 위한 langchain_chroma, PDF를 앍기 위한 pypdf를 설치합니다.

In [1]:
!pip install langchain_openai langchain_community langchain_chroma pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 59.7 MB/s eta 0:00

In [4]:
import os
import requests
from typing import List
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever

OpenAI의 키값을 셋팅 합니다.

In [5]:
os.environ['OPENAI_API_KEY'] = 'Openai_api_key'

### 3. 데이터 다운로드
데이터는 2023_ 북한인권보고서.pdf 입니다.

In [7]:
url = "https://github.com/llama-index-tutorial/llama-index-tutorial/raw/main/ch07/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf"
filename = "2023_북한인권보고서.pdf"

response = requests.get(url)
with open(filename, "wb") as f:
  f.write(response.content)

print(f"{filename} 다운 완료!")


2023_북한인권보고서.pdf 다운 완료!


###3. 거대 언어 모델과 임베딩 설정
랭체인을 사용하여 사용할 각종 설정들의 값을 정합니다.

In [9]:
# LangChain의 LLM과 임베딩 모델 설정
llm = ChatOpenAI(model='gpt-4o', temperature=0.2)
embed_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 문서 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 100)

# PDF 문서를 읽고 벡터 인덱스 생성
loader = PyPDFLoader("2023_북한인권보고서.pdf")
documents = loader.load()
chunks = text_splitter.split_documents(documents)
vector_store = Chroma.from_documents(chunks, embed_model)

문서처리를위한chunk_size=300과chunk_overlap=100설정은RecursiveCharacterTextSplitter
에서 적용됩니다. 길이 300 기준으로 문서를 나누고 길이 100 자의 문자열을 중복되게 하여, 나중에 문서를 검색할 때 문맥이 끊기지 않도록 합니다

이렇게설정한후, PyPDFLoader로PDF파일을읽어들이고load()로텍스트를추출합니다. 추출된텍스트는 text_splitter.split_documents()를 통해 분할되고, Chroma.from_documents
()를 통해 벡터 인덱스로 변환되는데, 이 과정에서 위에서 설정한 임베딩 모델과 청크 설정이 모두 적용됩니다.

###4. 리랭킹 구현하기
리랭킹을 사용하지 않는 기본 검색 엔진과 리랭킹을 사용하는 검색 엔진을 둘 다 별도로
구현하여 검색 결과의 차이를 확인해보겠습니다.

In [17]:
# 기본 검색 엔진 (리랭킹 없음)
basic_retriever = vector_store.as_retriever(search_kwargs = {"k":4})

# Reranker 설정
cross_encoder = HuggingFaceCrossEncoder(model_name = "BAAI/bge-reranker-v2-m3")
reranker = CrossEncoderReranker(model=cross_encoder, top_n=2)

# 리랭킹이 포함된 검색 엔진
rerank_retriever = ContextualCompressionRetriever(
    base_compressor = reranker,
    base_retriever = basic_retriever
)

# 최종 답변 생성 함수
def generate_answer(query:str, documents:List[Document]) ->str:
  context = "\n\n".join([doc.page_content for doc in documents])

  prompt = f"""다음 검색 결과를 바탕으로 질문에 답변해주세요.
  검색 결과의 정보를 최대한 사용하고, 없는 정보는 답변하지 마세요.

  검색결과:
  {context}

  질문: {query}

  답변:"""

  response = llm.invoke(prompt)
  return response.content

basic_retriever는 리랭킹을 사용하지 않는 기본적인 벡터 검색만 수행하는 리트리버로,
search_kwargs={"k": 4}는 임베딩 유사도를 기준으로 상위 4 개의 가장 관련성 높은 문서 조
각을 찾아내도록 설정합니다. 이 과정에서는 텍스트를 벡터로 변환하고 코사인 유사도를 계산하여 가장 가까운 문서들을 찾아냅니다. 이 리트리버는 단순하지만 빠른 검색이 가능합니다.

cross_encoder와 reranker 객체는 아래에서 구현할 리랭킹을 사용하는 검색엔진rerank_retriever에 서 리 랭 킹 을 위 해 서 사 용 될 크 로 스 인 코 더 모 델 입 니 다. BAAI/bgereranker-v2-m3 모델을 사용하여 문장 간의 의미적 관계를 더 깊이 이해하고 정확한 순위를 매깁니다. 크로스 인코더 방식이기 때문에 질문과 문서를 함께 분석하여, 단순한 벡터 유사도보다 더 정확한 관련성 판단이 가능하며, top_n=2 설정으로 가장 관련성 높은 2 개만 선택하도록 준비합니다.

rerank_retriever는 리랭킹을 사용하는 검색 엔진으로, basic_retriever와 동일한 벡터 검색
방식에위에서준비한reranker객체(즉,크로스인코더모델)를ContextualCompressionRetriever
를 통해 추가하여 더 정교한 검색을 구현합니다. 기본 리트리버가 4 개의 문서를 찾은 후 reranker 로 결과를 정제하여 더 정확한 검색 결과를 제공합니다. 이러한 이중 필터링이라는 리랭킹 과정을 통해 리랭킹을 사용하지 않는 basic_retriever 검색 대비 검색의 정확도를 향상시킬 수 있습니다. 하지만 리랭킹
과정 때문에 basic_retriever보다 더 많은 처리 시간을 필요로 합니다. 이는 기본 검색만 수행하는
basic_retriever와는 달리 기본 검색 이후 크로스 인코더 모델에 해당하는 reranker 객체를 한
번 더 사용하기 때문입니다.

우선 리랭킹을 사용하지 않는 검색 엔진 basic_retriever를 사용하였을 때의 답변과 답변에 참고한 검색 결과를 보겠습니다.


In [18]:
# 쿼리 실행
query = "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"

print("=== 기본 검색 엔진 검색 결과")
basic_documents = basic_retriever.invoke(query)
basic_response = generate_answer(query, basic_documents)

print(f"\n질문: {query}")
print(f"\n답변: {basic_response}")
print(f"\n검색된 문서:")
for i, doc in enumerate(basic_documents):
  print(f"\n검색 문서 {1+i}:")
  print(doc.page_content)
  print(("----"))

=== 기본 검색 엔진 검색 결과

질문: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식

답변: 2019년 말 평양시 소재 기업소에서 일하던 노동자는 매월 쌀 6㎏, 기름 5ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리를 배급받았다는 증언이 있습니다.

검색된 문서:

검색 문서 1:
화 또는 쌀이나 기름 등 현물로 지급하였다고 한다. 2019년 평양
의 외화벌이 사업소에서는 보수 50달러를 월 2회로 나누어 현금으
로 지급하였다고 하는 사례가 있었고, 평양 외화벌이 식당에서는 매
----

검색 문서 2:
파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, 
곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았
----

검색 문서 3:
가배급을 선택하고, 잘사는 기업소들은 기업소 자체 배급을 선택합
니 다. 세대주가 직장에 다닐 경우 세대주만 직장에서 배급을 받고 
가족들은 국가배급소에서 배급을 받습니다. 평양시와 자강도는 대
체로 다 줬는데 다른 지역은 배급이 잘 안되고 배급제가 없어졌다는 
소리를 들었습니다. ”
국가배급의 주기, 양, 곡물의 종류 등에서 평양시와 지방의 차이
가 크게 나고 있었다. 식량배급이 비교적 원활하게 작동하는 지역은 
평양시로 보이는데, 2017년 어머니가 지역배급 대상자로 배급표가
----

검색 문서 4:
한 달을 생활하기에 부족한 금액이었다고 하였다. 2018년 양강도의 
무역사업소에서는 1년치 노동 보수와 배급을 한 번에 지급하였다고 
하는데, 지급된 금액은 노동자 1명에게 1,800위안으로 약 300만원 
정도였다고 하였다.

query 변수에 “19 년 말 평양시 소재 기업소에서 달마다 배급받은 음식” 이라는 구체적인 질문을 저장합
니다. 이 질문은 시기 (19 년 말), 장소 (평양시), 대상 (기업소), 내용 (배급받은 음식) 이 모두 명시된 구체적인 검색 요청입니다. 이어서 basic_retriever.invoke(query)를 통해 기본 벡터 검색을 수행합니다. 이때 내부적으로는 입력된 질문을 OpenAI 의 임베딩 모델로 벡터화하고, Chroma 벡터 데이터베이스에서 코사인 유사도를 계산한 후, 유사도가 높은 상위 4 개 문서를 추출하여 basic_documents 변수에 저장합니다. 이 과정은 앞서 설정한 search_kwargs={"k": 4} 파라미터에 의해 제어됩니다.

검색이 완료되면 generate_answer(query, basic_documents)를 호출하여 검색된 문서들
을 바탕으로 최종 답변을 생성합니다. 이 함수 내부에서는 4 개의 검색된 문서 내용을 하나의 컨텍스트로 결합하고, “검색 결과를 바탕으로 답변하라” 는 지시사항과 함께 프롬프트를 구성한 후, GPT‑4o 모델에 전송하여 자연어 답변을 생성하고 basic_response 변수에 저장합니다.

마지막으로 입력된 질문, LLM 이 생성한 최종 답변, 검색에 사용된 4 개 문서의 내용을 출력합니다

기본 검색에서는 4 개의 문서가 모두 반환되었는데, 벡터 유사도 순위를 살펴보면 다음과 같습니다:
- 검색 문서 1: 2019 년 평양의 외화벌이 사업소 보수 관련 내용으로 “음식 배급” 과는 직접적인 관련이 없
습니다.
- 검색 문서 2: 2019 년 평양시 기업소의 구체적인 배급량이 명시된 가장 중요한 문서입니다. 질문에 부합
하는 내용입니다.
- 검색 문서 3: 평양시의 일반적인 배급 시스템에 대한 설명으로, 맥락은 관련있지만 구체적인 배급량 정
보는 없습니다.
- 검색 문서 4: 양강도 합영회사 관련 내용으로 지역과 기관 유형이 모두 다릅니다.

기본 벡터 검색에서는 가장 중요한 검색 문서 2 가 2 순위에 배치되었고, 상대적으로 관련성이
낮은 문서 1 이 1 순위에 올라온 것을 볼 수 있습니다. 이번에는 리랭킹을 사용하는 검색 엔진인
rerank_retriever를 사용하였을 때의 답변과 답변을 위해 참고한 검색 결과를 보겠습니다.

In [19]:
print("\n\n==== 리랭킹 후 검색결과 ====")
rerank_documents = rerank_retriever.invoke(query)
rerank_response = generate_answer(query, rerank_documents)

print(f"\n질문: {query}")
print(f"답변: {rerank_response}")
print("\n검색된 문서:")
for i, doc in enumerate(rerank_documents):
  print((f"\n검색 문서 {i+1}"))
  print(doc.page_content)
  print("------")



==== 리랭킹 후 검색결과 ====


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



질문: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식
답변: 2019년 말 평양시 소재 기업소에서 일하던 노동자는 매월 쌀 6㎏, 기름 5ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 등을 배급받았습니다.

검색된 문서:

검색 문서 1
파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, 
곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았
------

검색 문서 2
가배급을 선택하고, 잘사는 기업소들은 기업소 자체 배급을 선택합
니 다. 세대주가 직장에 다닐 경우 세대주만 직장에서 배급을 받고 
가족들은 국가배급소에서 배급을 받습니다. 평양시와 자강도는 대
체로 다 줬는데 다른 지역은 배급이 잘 안되고 배급제가 없어졌다는 
소리를 들었습니다. ”
국가배급의 주기, 양, 곡물의 종류 등에서 평양시와 지방의 차이
가 크게 나고 있었다. 식량배급이 비교적 원활하게 작동하는 지역은 
평양시로 보이는데, 2017년 어머니가 지역배급 대상자로 배급표가
------


크로스 인코더 리랭킹 후에는 문서 수가 4 개에서 2 개로 줄어들면서 리랭킹을 사용하지 않는 기본 검색에
서 2 순위였던 가장 중요한 문서 (구체적 배급량 정보) 가 1 순위로 올라왔습니다. 그 외에 문서 수가 4 개
에서 2 개로 줄어드는 과정에서 외화벌이 사업소 현금 지급 관련 문서 (기본 검색 문서 1) 와 양강도 합영
회사 관련 문서 (기본 검색 문서 4) 가 제거되었습니다. 결론적으로 리랭킹 후에는 실제 답변에 사용되는
핵심 문서 (기존 검색 엔진에서의 검색 문서 2 번) 가 최고 순위인 검색 문서 1 번으로 올라온 것을 확인할
수 있습니다.
이처럼 리랭킹은 기존의 검색 엔진에서 실제 질문과 연관된 문서들을 필터링 할 수 있는 더 강력한 모델을
두번째 검색 모델로 두어서 검색 성능을 높이므로서 답변에서의 오류를 줄일 수 있는 방법입니다. 이어서
검색 성능을 높일 수 있는 또 다른 방법인 하이드 (HyDE) 에 대해서 알아봅시다.